# Chain Prompting with Rotten Tomatoes Reviews

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [24]:
import os
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

In [36]:
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [26]:
df = pd.read_csv('rotten_tomatoes_critic_reviews.csv')

In [27]:
df.head()

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...


In [29]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [30]:
llm = ChatOpenAI(temperature=0.2, model="gpt-3.5-turbo")

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in less than 10 words:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="Summary"
                    )


In [31]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you translate the following sentence into French:"
    "\n\n{Summary}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="Translation"
                    )

In [32]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two],
    input_variables=["Review"],
    output_variables=["Summary","Translation"],
    verbose=True
)

In [35]:
review = df.review_content[3]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Whether audiences will get behind The Lightning Thief is hard to predict. Overall, it's an entertaining introduction to a promising new world -- but will the consuming shadow of Potter be too big to break free of?",
 'Summary': 'Entertaining introduction to a promising world, overshadowed by Potter.',
 'Translation': 'Introduction divertissante à un monde prometteur, éclipsée par Potter.'}